## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [2]:
import pandas as pd
import numpy as np

In [18]:
#1) Import N rows from the data file

N = 200000 #value between 10k and 1310720 (nr of rows)
file_name = "~/data/data_000637.txt"
data = pd.read_csv(file_name, nrows=N)
data #to check

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
199995,1,0,139,3869202157,76,0
199996,1,0,62,3869202157,76,2
199997,1,1,2,3869202157,94,15
199998,1,1,1,3869202157,96,13


In [19]:
#2) Find 'x': max of BX_COUNTER+1. 
#assumption: N large enough that the max is reached within the N samples

xmax = data.loc[:,'BX_COUNTER'].max() + 1
print('In each orbit there are', xmax, 'bx')

In each orbit there are 3564 bx


In [20]:
#3) How much the data taking lasted:

#start time
t0 = [data.loc[0,'ORBIT_CNT'], data.loc[0, 'BX_COUNTER'], data.loc[0, 'TDC_MEAS']]
#end time of sample
tT = [data.loc[N-1,'ORBIT_CNT'], data.loc[N-1, 'BX_COUNTER'], data.loc[N-1, 'TDC_MEAS']]
#conversion
aq_time = (tT[2]-t0[2])*25/30 + (tT[1]-t0[1]+(tT[0]-t0[0])*xmax)*25 
print(N, "samples were acquired in %.2f ns" % aq_time)
#inference
total_time = aq_time*1310720/N
print("The whole dataset was acquired in %.2f ns" % total_time)

200000 samples were acquired in 177252319.17 ns
The whole dataset was acquired in 1161640798.89 ns


In [6]:
# 4) New column with the absolute time in ns

data['time_ns'] = (data['TDC_MEAS']-t0[2])*25/30 + (data['BX_COUNTER']-t0[1]+(data['ORBIT_CNT']-t0[0])*xmax)*25
data #to check

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.333333e-01
2,1,0,63,3869200167,2553,28,4.476667e+03
3,1,0,64,3869200167,2558,19,4.594167e+03
4,1,0,64,3869200167,2760,25,9.649167e+03
...,...,...,...,...,...,...,...
199995,1,0,139,3869202157,76,0,1.772515e+08
199996,1,0,62,3869202157,76,2,1.772515e+08
199997,1,1,2,3869202157,94,15,1.772520e+08
199998,1,1,1,3869202157,96,13,1.772520e+08


In [7]:
#5) Replacing the values of the HEAD column randomly with 0 or 1.

data.loc[:,'HEAD'] = np.random.randint(2, size=N)
data #to check

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.333333e-01
2,0,0,63,3869200167,2553,28,4.476667e+03
3,1,0,64,3869200167,2558,19,4.594167e+03
4,0,0,64,3869200167,2760,25,9.649167e+03
...,...,...,...,...,...,...,...
199995,1,0,139,3869202157,76,0,1.772515e+08
199996,0,0,62,3869202157,76,2,1.772515e+08
199997,0,1,2,3869202157,94,15,1.772520e+08
199998,0,1,1,3869202157,96,13,1.772520e+08


In [8]:
#6. Create a new DataFrame that contains only the rows with HEAD=1.

datah1 = data[data['HEAD'] == 1]
datah1 #to check

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.333333e-01
3,1,0,64,3869200167,2558,19,4.594167e+03
5,1,0,63,3869200167,2762,4,9.681667e+03
6,1,0,61,3869200167,2772,14,9.940000e+03
...,...,...,...,...,...,...,...
199990,1,0,74,3869202157,33,11,1.772505e+08
199991,1,0,74,3869202157,41,11,1.772507e+08
199992,1,0,64,3869202157,61,24,1.772512e+08
199994,1,0,61,3869202157,74,2,1.772515e+08


In [9]:
#7. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

#filter data of the FPGA 0
dataf0 = data[data['FPGA'] == 0]
hist0 = dataf0.hist(column='TDC_CHANNEL')

#filter data of the FPGA 1
dataf1 = data[data['FPGA'] == 1]
hist1 = dataf1.hist(column='TDC_CHANNEL')

In [13]:
#8. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

#find number of occurencied of each TDC channel
gb = data.groupby('TDC_CHANNEL').sum()
#Sorting appropriately
sgb = gb.sort_values(by="HEAD", ascending = False)
print('The three most noisy channels are:', sgb.index[0], sgb.index[1], sgb.index[2])

The three most noisy channels are: 139 64 63


In [14]:
#9. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

#exploiting pandas function unique
uniques = pd.unique(data.loc[:,'ORBIT_CNT'])
print('The number of unique orbits is', len(uniques))

#filtering data of TDC channel 139
data139 = data[data['TDC_CHANNEL'] == 139]
uniques139 = pd.unique(data139.loc[:,'ORBIT_CNT'])
print('The number of unique orbits with at least one measurement from TDC_CHANNEL 139 is', len(uniques139))

The number of unique orbits is 1989
The number of unique orbits with at least one measurement from TDC_CHANNEL 139 is 1987
